## General

In [1]:
import os
from Crypto.Protocol.KDF import PBKDF2 
from Crypto.Cipher import AES
from pynq import Overlay
from pynq import allocate
import numpy as np
import time
import gc

In [2]:
def prefix(num, decimal=2):
    if decimal<0:
        strformat = '%f'
    else:
        strformat = '%%.%df'%(decimal)
    if num>=1000000000000:
        return (strformat+' T')%(num/1000000000000)
    elif num>=1000000000:
        return (strformat+' G')%(num/1000000000)
    elif num>=1000000:
        return (strformat+' M')%(num/1000000)
    elif num>=1000:
        return (strformat+' k')%(num/1000)
    else:
        return '%s'%(num)

In [3]:
def pass_fail(condition):
    return 'PASS' if condition else 'FAILED'

###### Mount USB drive

In [4]:
%run ./linux_usb_mount.ipynb

/mnt/usb


######  Generate an AES key

In [5]:
#AES Utils
%run ./AES_Utils.ipynb

In [6]:
print(key)

b'\x97\xc6\x17\x85\xe7(\xa6\x01\xb8\xd7\x1d\xa4L\xb3\x0b\x1c#\xff\xfem\x12\x9d%\xcc_\xd7hVb\\\x8d\xfa'


In [7]:
%run ./AES_Cores.ipynb

In [8]:
# Testing parameters

In [9]:
# TEST_GB_001 TEST_GB_002
# TEST_MB_001 TEST_MB_005 TEST_MB_010 TEST_MB_050 TEST_MB_100 TEST_MB_500
# TEST_kB_001 TEST_kB_005 TEST_kB_010 TEST_kB_050 TEST_kB_100 TEST_kB_500

In [10]:
encrypt_file_source = os.path.join(usb_path+'/testFile/', "TEST_kB_001")

In [11]:
encrypt_file_rename = encrypt_file_source+".rename"

## Software

###### Configuration

In [12]:
skip_software = False

###### Free memory

In [13]:
gc.collect()

120

###### Encrypt / Decrypt file

In [14]:
encrypt_extention = '.ECRYP'

In [15]:
def encryptFile(fileName, key, rename=None, batch_size_log2=24):
    batch_size = int(2**batch_size_log2)
    get_file_size = os.path.getsize(fileName)
    file_size = get_file_size
    if rename is None:
        rename = fileName+encrypt_extention
    try:
        fr = open(fileName, 'rb')
        fw = open(rename, 'wb')
        #fw.write(b'ECRYP')
        #fw.write(bytes([batch_size_log2]))
        
        while file_size>0:
            content = fr.read(batch_size)
            if file_size <= batch_size:
                content = aesEncrypt(content, key, padding=True)
            else:
                content = aesEncrypt(content, key, padding=False)
            fw.write(content)
            file_size = file_size-batch_size
    finally:
        if fr:
            fr.close()
        if fw:
            fw.close()
    
    return get_file_size

In [16]:
def decryptFile(fileName, key, rename=None, batch_size_log2=24):
    if encrypt_extention == fileName[-len(encrypt_extention):]:
        get_file_size = os.path.getsize(fileName)
        file_size = get_file_size
        if rename is None:
            rename = fileName[:-len(encrypt_extention)]
        try:
            fr = open(fileName, 'rb')
            #if fr.read(5) != b'ECRYP':
            #    fr.close()
            #    return False
            #batch_size_log2 = int.from_bytes(fr.read(1), byteorder='big')

            batch_size = 2**batch_size_log2
            fw = open(rename, 'wb')

            while file_size>0:
                content = fr.read(batch_size)
                if file_size <= batch_size:
                    content = aesDecrypt(content, key, padding=True)
                else:
                    content = aesDecrypt(content, key, padding=False)
                fw.write(content)
                file_size = file_size-batch_size
        finally:
            if fr:
                fr.close()
            if fw:
                fw.close()
                
        return True
    else:
        return False

###### Testing

In [17]:
print("Encrypting: %s"%(encrypt_file_source))

sum_se_filesize = os.path.getsize(encrypt_file_source)
print("File Size: %sB"%(prefix(sum_se_filesize, decimal=2)))

time_start = time.time()
if not skip_software:
    encryptFile(encrypt_file_source, key, rename = encrypt_file_rename+encrypt_extention)
time_stop = time.time()

sum_se_duration = time_stop - time_start
print("Duration: %f sec"%(sum_se_duration))

print("Encrypt done!\n")

Encrypting: /mnt/usb/testFile/TEST_kB_001
File Size: 1.02 kB
Duration: 0.005705 sec
Encrypt done!



In [18]:
print("Decrypting: %s"%(encrypt_file_rename+encrypt_extention))

if not skip_software:
    sum_sd_filesize = os.path.getsize(encrypt_file_rename+encrypt_extention)
else:
    sum_sd_filesize = 0
print("File Size: %sB"%(prefix(sum_se_filesize, decimal=2)))

time_start = time.time()
if not skip_software:
    decryptFile(encrypt_file_rename+encrypt_extention, key)
time_stop = time.time()

sum_sd_duration = time_stop - time_start
print("Duration: %f sec"%(sum_sd_duration))

print("Decrypt done!\n")

Decrypting: /mnt/usb/testFile/TEST_kB_001.rename.ECRYP
File Size: 1.02 kB
Duration: 0.008269 sec
Decrypt done!



###### Checksum

In [19]:
if not skip_software:
    sum_so_md5 = cmd("md5sum %s"%(encrypt_file_source))
    print(sum_so_md5)
    sum_so_md5 = sum_so_md5.split(" ")[0]
    sum_se_md5 = cmd("md5sum %s"%(encrypt_file_rename+encrypt_extention))
    print(sum_se_md5)
    sum_se_md5 = sum_se_md5.split(" ")[0]
    sum_sd_md5 = cmd("md5sum %s"%(encrypt_file_rename))
    print(sum_sd_md5)
    sum_sd_md5 = sum_sd_md5.split(" ")[0]
else:
    sum_so_md5='SKIP-000'
    sum_se_md5='SKIP-001'
    sum_sd_md5='SKIP-002'

a7c12e120cc47e5c11efa6b837f9df93  /mnt/usb/testFile/TEST_kB_001

04a3012cc58771095c306a8d7cd8299d  /mnt/usb/testFile/TEST_kB_001.rename.ECRYP

a7c12e120cc47e5c11efa6b837f9df93  /mnt/usb/testFile/TEST_kB_001.rename



###### Delete files

In [20]:
print(cmd("rm -f %s"%(encrypt_file_rename+encrypt_extention)))
print(cmd("rm -f %s"%(encrypt_file_rename)))

## Hardware

###### Configuration

In [21]:
hardware_encrypt_overlay = "./bit/encrypt_aes256_150M_core6.bit"
hardware_decrypt_overlay = "./bit/decrypt_aes256_150M_core6.bit"
hardware_core_count = 6
hardware_block_size = 2**23
hardware_self_test = True

###### Free memory

In [22]:
gc.collect()

20

#### Encrypt file

In [23]:
ol = Overlay(hardware_encrypt_overlay)

In [24]:
core_number = hardware_core_count
block_size = hardware_block_size

cores = [ AES_Core( 
    ol.__getattr__(f"AES_ECB_encrypt_{i}") ,
    ol.__getattr__(f"axi_dma_write_{i}") ,
    ol.__getattr__(f"axi_dma_read_{i}"),
    key ,
    block_size,
    self_test = hardware_self_test
) for i in range(core_number)]

AES_ECB_encrypt_0 self encrypt test success
AES_ECB_encrypt_1 self encrypt test success
AES_ECB_encrypt_2 self encrypt test success
AES_ECB_encrypt_3 self encrypt test success
AES_ECB_encrypt_4 self encrypt test success
AES_ECB_encrypt_5 self encrypt test success


In [25]:
aes_hw = AES_Schedule( cores,  block_size = block_size )

Init success with 6 AES cores


###### Testing

In [26]:
sum_he_filesize = os.path.getsize(encrypt_file_source)
print("File Size: %sB"%(prefix(sum_he_filesize, decimal=2)))
start_time = time.time()
aes_hw.enqueue_file(file=encrypt_file_source)
aes_hw.start_encrypt_schedule()
end_time = time.time()
sum_he_duration = end_time - start_time
print(f"time: {sum_he_duration}")

File Size: 1.02 kB
start encrypt /mnt/usb/testFile/TEST_kB_001 with size 1024
need do 0 time

time: 1.4670977592468262


###### Free memory

In [27]:
del ol
del aes_hw
gc.collect()

60

#### Decrypt file

In [28]:
ol = Overlay(hardware_decrypt_overlay)

In [29]:
core_number = hardware_core_count
block_size = hardware_block_size

for i,core in enumerate(cores):
    core.init(
        ol.__getattr__(f"AES_ECB_decrypt_{i}") ,
        ol.__getattr__(f"axi_dma_write_{i}") ,
        ol.__getattr__(f"axi_dma_read_{i}"),
        key ,
        self_test = hardware_self_test
    )

AES_ECB_decrypt_0 self decrypt test success
AES_ECB_decrypt_1 self decrypt test success
AES_ECB_decrypt_2 self decrypt test success
AES_ECB_decrypt_3 self decrypt test success
AES_ECB_decrypt_4 self decrypt test success
AES_ECB_decrypt_5 self decrypt test success


In [30]:
aes_hw = AES_Schedule( cores,  block_size = block_size )

Init success with 6 AES cores


###### Testing

In [31]:
sum_hd_filesize = os.path.getsize(encrypt_file_source)
print("File Size: %sB"%(prefix(sum_hd_filesize, decimal=2)))
start_time = time.time()
aes_hw.enqueue_file(file=encrypt_file_source+'.encrypt')
aes_hw.start_decrypt_schedule()
end_time = time.time()
sum_hd_duration = end_time - start_time
print(f"time: {sum_hd_duration}")

File Size: 1.02 kB
start decrypt /mnt/usb/testFile/TEST_kB_001.encrypt with size 1040
need do 0 time

time: 1.4555861949920654


###### Checksum

In [32]:
sum_ho_md5 = cmd("md5sum %s"%(encrypt_file_source))
print(sum_ho_md5)
sum_ho_md5 = sum_ho_md5.split(" ")[0]
sum_he_md5 = cmd("md5sum %s"%(encrypt_file_source+'.encrypt'))
print(sum_he_md5)
sum_he_md5 = sum_he_md5.split(" ")[0]
sum_hd_md5 = cmd("md5sum %s"%(encrypt_file_source+'.encrypt'+'.decrypt'))
print(sum_hd_md5)
sum_hd_md5 = sum_hd_md5.split(" ")[0]

a7c12e120cc47e5c11efa6b837f9df93  /mnt/usb/testFile/TEST_kB_001

04a3012cc58771095c306a8d7cd8299d  /mnt/usb/testFile/TEST_kB_001.encrypt

a7c12e120cc47e5c11efa6b837f9df93  /mnt/usb/testFile/TEST_kB_001.encrypt.decrypt



###### Delete files

In [33]:
print(cmd("rm -f %s"%(encrypt_file_source+'.encrypt')))
print(cmd("rm -f %s"%(encrypt_file_source+'.encrypt'+'.decrypt')))

###### Free memory

In [34]:
del ol
del aes_hw
gc.collect()

148

## =========================================

## Summary

#### Information

In [35]:
print('%-20s %-80s'%('Target File', encrypt_file_source))
print('%-20s %-80s'%('File Size', prefix(os.path.getsize(encrypt_file_source),decimal=-1)+'B'))

Target File          /mnt/usb/testFile/TEST_kB_001                                                   
File Size            1.024000 kB                                                                     


#### Performance

In [36]:
print('%-20s %-15s %-15s %-15s %-15s'%('', 'Software EN', 'Software DE', 'Hardware EN', 'Hardware DE'))
print('%-20s %-15s %-15s %-15s %-15s'%('Handle File Size', prefix(sum_se_filesize,decimal=-1)+'B', prefix(sum_sd_filesize,decimal=-1)+'B', prefix(sum_he_filesize,decimal=-1)+'B', prefix(sum_hd_filesize,decimal=-1)+'B'))
print('%-20s %-15.4f %-15.4f %-15.4f %-15.4f'%('Duration', sum_se_duration, sum_sd_duration, sum_he_duration, sum_hd_duration))
print('%-20s %-15s %-15s %-15s %-15s'%('Bytes/s', prefix(sum_se_filesize/sum_se_duration)+'B/s', prefix(sum_sd_filesize/sum_sd_duration)+'B/s', prefix(sum_he_filesize/sum_he_duration)+'B/s', prefix(sum_hd_filesize/sum_hd_duration)+'B/s'))

                     Software EN     Software DE     Hardware EN     Hardware DE    
Handle File Size     1.024000 kB     1.040000 kB     1.024000 kB     1.024000 kB    
Duration             0.0057          0.0083          1.4671          1.4556         
Bytes/s              179.50 kB/s     125.77 kB/s     697.9766641629237B/s 703.4966417812049B/s


#### Correctness

In [37]:
print('%-15s %-35s %-35s %-7s'%('MD5 Checksum', 'Software', 'Hardware', ''))
print('%-15s %-35s %-35s %-7s'%('Source File', sum_so_md5, sum_ho_md5, pass_fail(sum_so_md5==sum_ho_md5)))
print('%-15s %-35s %-35s %-7s'%('Encrypted', sum_se_md5, sum_he_md5, pass_fail(sum_se_md5==sum_he_md5)))
print('%-15s %-35s %-35s %-7s'%('Recovery', sum_sd_md5, sum_hd_md5, pass_fail(sum_sd_md5==sum_hd_md5)))
print('%-15s %-35s %-35s %-7s'%('', pass_fail(sum_so_md5==sum_sd_md5), pass_fail(sum_ho_md5==sum_hd_md5), ''))

MD5 Checksum    Software                            Hardware                                   
Source File     a7c12e120cc47e5c11efa6b837f9df93    a7c12e120cc47e5c11efa6b837f9df93    PASS   
Encrypted       04a3012cc58771095c306a8d7cd8299d    04a3012cc58771095c306a8d7cd8299d    PASS   
Recovery        a7c12e120cc47e5c11efa6b837f9df93    a7c12e120cc47e5c11efa6b837f9df93    PASS   
                PASS                                PASS                                       


## =========================================

In [38]:
# TEST_GB_001 TEST_GB_002
# TEST_MB_001 TEST_MB_005 TEST_MB_010 TEST_MB_050 TEST_MB_100 TEST_MB_500
# TEST_kB_001 TEST_kB_005 TEST_kB_010 TEST_kB_050 TEST_kB_100 TEST_kB_500

In [47]:
#soft_skip_condition=['TEST_GB_002','TEST_GB_001', 'TEST_MB_500', 'TEST_MB_100', 'TEST_MB_050', 'TEST_MB_010', 'TEST_MB_005', 'TEST_MB_001']
soft_skip_condition=['TEST_GB_002','TEST_GB_001']

# Software Batch

In [48]:
encrypt_extention = '.en'
batch_size_log2 = 23

In [50]:
def batch_software_en(path = r'/mnt/usb/testFile'):
    filesize_list=[]
    duration_list=[]
    for root, dirs, files in os.walk(path):
        for fileName in files:
            if fileName in soft_skip_condition:
                continue
            if ".goal"+encrypt_extention in fileName:
                continue
            print("-----------------------------------------")
            full_file_name = os.path.join(root, fileName)
            print("Encrypting: %s"%(full_file_name))
            sum_se_filesize = os.path.getsize(full_file_name)
            print("File Size: %sB"%(prefix(sum_se_filesize, decimal=2)))
            time_start = time.time()
            encryptFile(full_file_name, key, rename = full_file_name+".goal"+encrypt_extention, batch_size_log2=batch_size_log2)
            time_stop = time.time()
            sum_se_duration = time_stop - time_start
            print("Duration: %f sec"%(sum_se_duration))
            filesize_list.append(sum_se_filesize)
            duration_list.append(sum_se_duration)
    print("-----------------------------------------")
    print("Encrypt done!\n")
    print(filesize_list)
    print(duration_list)
batch_software_en()


-----------------------------------------
Encrypting: /mnt/usb/testFile/TEST_kB_001
File Size: 1.02 kB
Duration: 0.002596 sec
-----------------------------------------
Encrypting: /mnt/usb/testFile/TEST_kB_005
File Size: 5.12 kB
Duration: 0.004682 sec
-----------------------------------------
Encrypting: /mnt/usb/testFile/TEST_kB_010
File Size: 10.24 kB
Duration: 0.006776 sec
-----------------------------------------
Encrypting: /mnt/usb/testFile/TEST_kB_050
File Size: 51.20 kB
Duration: 0.010499 sec
-----------------------------------------
Encrypting: /mnt/usb/testFile/TEST_kB_100
File Size: 102.40 kB
Duration: 0.019850 sec
-----------------------------------------
Encrypting: /mnt/usb/testFile/TEST_kB_500
File Size: 512.00 kB
Duration: 0.087920 sec
-----------------------------------------
Encrypting: /mnt/usb/testFile/TEST_MB_001
File Size: 1.05 MB
Duration: 0.181009 sec
-----------------------------------------
Encrypting: /mnt/usb/testFile/TEST_MB_005
File Size: 5.24 MB
Duration:

In [57]:
def batch_software_en_batchsize(path = r'/mnt/usb/testFile/TEST_MB_001'):
    batchsize = [23,22,21,20,19,18,17]
    filesize_list=[]
    duration_list=[]
    for batch_size_log in batchsize:
        print("-----------------------------------------")
        full_file_name = path
        print("batch_size_log: %s"%(batch_size_log))
        sum_se_filesize = os.path.getsize(full_file_name)
        print("File Size: %sB"%(prefix(sum_se_filesize, decimal=2)))
        time_start = time.time()
        encryptFile(full_file_name, key, rename = full_file_name+".goal"+encrypt_extention, batch_size_log2=batch_size_log)
        time_stop = time.time()
        sum_se_duration = time_stop - time_start
        print("Duration: %f sec"%(sum_se_duration))
        filesize_list.append(sum_se_filesize)
        duration_list.append(sum_se_duration)
    print("-----------------------------------------")
    print("Encrypt done!\n")
    print(filesize_list)
    print(duration_list)
batch_software_en_batchsize()


-----------------------------------------
batch_size_log: 23
File Size: 1.05 MB
Duration: 0.183525 sec
-----------------------------------------
batch_size_log: 22
File Size: 1.05 MB
Duration: 0.143247 sec
-----------------------------------------
batch_size_log: 21
File Size: 1.05 MB
Duration: 0.142121 sec
-----------------------------------------
batch_size_log: 20
File Size: 1.05 MB
Duration: 0.143171 sec
-----------------------------------------
batch_size_log: 19
File Size: 1.05 MB
Duration: 0.136430 sec
-----------------------------------------
batch_size_log: 18
File Size: 1.05 MB
Duration: 0.131856 sec
-----------------------------------------
batch_size_log: 17
File Size: 1.05 MB
Duration: 0.138259 sec
-----------------------------------------
Encrypt done!

[1048576, 1048576, 1048576, 1048576, 1048576, 1048576, 1048576]
[0.18352532386779785, 0.14324665069580078, 0.1421213150024414, 0.14317083358764648, 0.1364302635192871, 0.13185572624206543, 0.1382594108581543]


In [44]:
def batch_software_de(path = r'/mnt/usb/testFile'):
    filesize_list=[]
    duration_list=[]
    for root, dirs, files in os.walk(path):
        for fileName in files:
            if ".goal"+encrypt_extention not in fileName:
                continue
            print("-----------------------------------------")
            full_file_name = os.path.join(root, fileName)
            print("Decrypting: %s"%(full_file_name))
            sum_se_filesize = os.path.getsize(full_file_name)
            print("File Size: %sB"%(prefix(sum_se_filesize, decimal=2)))
            time_start = time.time()
            decryptFile(full_file_name, key, batch_size_log2=batch_size_log2)
            time_stop = time.time()
            sum_se_duration = time_stop - time_start
            print("Duration: %f sec"%(sum_se_duration))
            filesize_list.append(sum_se_filesize)
            duration_list.append(sum_se_duration)
    print("-----------------------------------------")
    print("Decrypt done!\n")
    print(filesize_list)
    print(duration_list)
batch_software_de()

-----------------------------------------
Decrypting: /mnt/usb/testFile/TEST_GB_001.goal.en
File Size: 1.07 GB
Duration: 265.179558 sec
-----------------------------------------
Decrypting: /mnt/usb/testFile/TEST_kB_001.goal.en
File Size: 1.04 kB
Duration: 0.035326 sec
-----------------------------------------
Decrypting: /mnt/usb/testFile/TEST_kB_005.goal.en
File Size: 5.14 kB
Duration: 0.022735 sec
-----------------------------------------
Decrypting: /mnt/usb/testFile/TEST_kB_010.goal.en
File Size: 10.26 kB
Duration: 0.026287 sec
-----------------------------------------
Decrypting: /mnt/usb/testFile/TEST_kB_050.goal.en
File Size: 51.22 kB
Duration: 0.034928 sec
-----------------------------------------
Decrypting: /mnt/usb/testFile/TEST_kB_100.goal.en
File Size: 102.42 kB
Duration: 0.036724 sec
-----------------------------------------
Decrypting: /mnt/usb/testFile/TEST_kB_500.goal.en
File Size: 512.02 kB
Duration: 0.176370 sec
-----------------------------------------
Decrypting: 

In [45]:
def batch_md5(path = r'/mnt/usb/testFile'):
    for root, dirs, files in os.walk(path):
        for fileName in files:
            if fileName in soft_skip_condition:
                continue
            print(cmd('md5sum %s'%(os.path.join(root, fileName))))
#batch_md5()

In [70]:
r=cmd('rm /mnt/usb/testFile/*.goal*')

# Hardware Batch

###### Free memory

In [58]:
gc.collect()

430

###### Configuration

In [59]:
hardware_encrypt_overlay = "./bit/encrypt_aes256_150M_core6.bit"
hardware_decrypt_overlay = "./bit/decrypt_aes256_150M_core6.bit"
hardware_core_count = 6
hardware_block_size = 2**23
hardware_self_test = True
hard_skip_condition=['TEST_GB_002']
#hard_skip_condition=['TEST_GB_002','TEST_GB_001', 'TEST_MB_500', 'TEST_MB_100', 'TEST_MB_050', 'TEST_MB_010', 'TEST_MB_005', 'TEST_MB_001']

#### Encrypt file

###### Free memory

In [60]:
gc.collect()

40

#### Encrypt file

In [61]:
ol = Overlay(hardware_encrypt_overlay)

In [62]:
core_number = hardware_core_count
block_size = hardware_block_size

for i,core in enumerate(cores):
    core.init(
        ol.__getattr__(f"AES_ECB_encrypt_{i}") ,
        ol.__getattr__(f"axi_dma_write_{i}") ,
        ol.__getattr__(f"axi_dma_read_{i}"),
        key ,
        self_test = hardware_self_test
    )

AES_ECB_encrypt_0 self encrypt test success
AES_ECB_encrypt_1 self encrypt test success
AES_ECB_encrypt_2 self encrypt test success
AES_ECB_encrypt_3 self encrypt test success
AES_ECB_encrypt_4 self encrypt test success
AES_ECB_encrypt_5 self encrypt test success


In [63]:
aes_hw = AES_Schedule( cores,  block_size = block_size )

Init success with 6 AES cores


In [72]:
def batch_hardware_en(path = r'/mnt/usb/testFile'):
    filesize_list=[]
    duration_list=[]
    for root, dirs, files in os.walk(path):
        for fileName in files:
            if fileName in hard_skip_condition:
                continue
            if ".encrypt" in fileName:
                continue
            print("-----------------------------------------")
            aes_hw = AES_Schedule( cores,  block_size = block_size )
            full_file_name = os.path.join(root, fileName)
            print("Encrypting: %s"%(full_file_name))
            sum_he_filesize = os.path.getsize(full_file_name)
            print("File Size: %sB"%(prefix(sum_he_filesize, decimal=2)))
            time_start = time.time()
            aes_hw.enqueue_file(file=full_file_name)
            aes_hw.start_encrypt_schedule()
            time_stop = time.time()
            sum_he_duration = time_stop - time_start
            print("Duration: %f sec"%(sum_he_duration))
            filesize_list.append(sum_he_filesize)
            duration_list.append(sum_he_duration)
    print("-----------------------------------------")
    print("Encrypt done!\n")
    print(filesize_list)
    print(duration_list)
batch_hardware_en()

-----------------------------------------
Init success with 6 AES cores
Encrypting: /mnt/usb/testFile/TEST_GB_001
File Size: 1.07 GB
start encrypt /mnt/usb/testFile/TEST_GB_001 with size 1073741824
need do 128 time
Start part  128 /  128 on core #1
Duration: 136.845079 sec
-----------------------------------------
Init success with 6 AES cores
Encrypting: /mnt/usb/testFile/TEST_kB_001
File Size: 1.02 kB
start encrypt /mnt/usb/testFile/TEST_kB_001 with size 1024
need do 0 time

Duration: 1.854695 sec
-----------------------------------------
Init success with 6 AES cores
Encrypting: /mnt/usb/testFile/TEST_kB_005
File Size: 5.12 kB
start encrypt /mnt/usb/testFile/TEST_kB_005 with size 5120
need do 0 time

Duration: 1.450066 sec
-----------------------------------------
Init success with 6 AES cores
Encrypting: /mnt/usb/testFile/TEST_kB_010
File Size: 10.24 kB
start encrypt /mnt/usb/testFile/TEST_kB_010 with size 10240
need do 0 time

Duration: 1.449399 sec
-------------------------------

In [73]:
def batch_hardware_en_batchsize(path = r'/mnt/usb/testFile/TEST_MB_100'):
    batchsize = [23,22,21,20,19,18,17]
    filesize_list=[]
    duration_list=[]
    for batch_size_log in batchsize:
        
        print("-----------------------------------------")
        aes_hw = AES_Schedule( cores,  block_size = 2**batch_size_log )
        full_file_name = path
        print("batch_size_log: %s"%(batch_size_log))
        sum_se_filesize = os.path.getsize(full_file_name)
        print("File Size: %sB"%(prefix(sum_se_filesize, decimal=2)))
        time_start = time.time()
        aes_hw.enqueue_file(file=full_file_name)
        aes_hw.start_encrypt_schedule()
        time_stop = time.time()
        sum_se_duration = time_stop - time_start
        print("Duration: %f sec"%(sum_se_duration))
        filesize_list.append(sum_se_filesize)
        duration_list.append(sum_se_duration)
        r=cmd('rm /mnt/usb/testFile/*.encrypt*')
    print("-----------------------------------------")
    print("Encrypt done!\n")
    print(filesize_list)
    print(duration_list)
batch_hardware_en_batchsize()

-----------------------------------------
Init success with 6 AES cores
batch_size_log: 23
File Size: 104.86 MB
start encrypt /mnt/usb/testFile/TEST_MB_100 with size 104857600
need do 12 time
Start part   12 /   12 on core #5
Duration: 9.189973 sec
-----------------------------------------
Init success with 6 AES cores
batch_size_log: 22
File Size: 104.86 MB
start encrypt /mnt/usb/testFile/TEST_MB_100 with size 104857600
need do 25 time
Start part   25 /   25 on core #0
Duration: 10.217056 sec
-----------------------------------------
Init success with 6 AES cores
batch_size_log: 21
File Size: 104.86 MB
start encrypt /mnt/usb/testFile/TEST_MB_100 with size 104857600
need do 50 time
Start part   50 /   50 on core #1
Duration: 17.532403 sec
-----------------------------------------
Init success with 6 AES cores
batch_size_log: 20
File Size: 104.86 MB
start encrypt /mnt/usb/testFile/TEST_MB_100 with size 104857600
need do 100 time
Start part  100 /  100 on core #3
Duration: 26.851746 sec


###### Free memory

In [ ]:
del ol
del aes_hw
gc.collect()

#### Decrypt file

In [ ]:
ol = Overlay(hardware_decrypt_overlay)

In [ ]:
core_number = hardware_core_count
block_size = hardware_block_size

for i,core in enumerate(cores):
    core.init(
        ol.__getattr__(f"AES_ECB_decrypt_{i}") ,
        ol.__getattr__(f"axi_dma_write_{i}") ,
        ol.__getattr__(f"axi_dma_read_{i}"),
        key ,
        self_test = hardware_self_test
    )

In [ ]:
aes_hw = AES_Schedule( cores,  block_size = block_size )

In [ ]:
def batch_hardware_de(path = r'/mnt/usb/testFile'):
    filesize_list=[]
    duration_list=[]
    for root, dirs, files in os.walk(path):
        for fileName in files:
            if '.encrypt' not in fileName:
                continue
            print("-----------------------------------------")
            full_file_name = os.path.join(root, fileName)
            print("Decrypting: %s"%(full_file_name))
            sum_hd_filesize = os.path.getsize(full_file_name)
            print("File Size: %sB"%(prefix(sum_hd_filesize, decimal=2)))
            time_start = time.time()
            aes_hw.enqueue_file(file=full_file_name)
            aes_hw.start_decrypt_schedule()
            time_stop = time.time()
            sum_hd_duration = time_stop - time_start
            print("Duration: %f sec"%(sum_hd_duration))
            filesize_list.append(sum_hd_filesize)
            duration_list.append(sum_hd_duration)
    print("-----------------------------------------")
    print("Decrypt done!\n")
    print(filesize_list)
    print(duration_list)
batch_hardware_de()

In [69]:
r=cmd('rm /mnt/usb/testFile/*.encrypt*')

###### Free memory

In [ ]:
del ol
del aes_hw
gc.collect()